In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services. In this notebook, we use ChatGPT, Gemini on GenerativeAI and Anthropic Claude.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# How to input langrila's modules

In langrila, universal message class allows you to handle each client message in a same way.

# Import modules

In [3]:
from langrila.claude import Claude
from langrila.gemini import Gemini
from langrila.openai import ChatGPT

# Define tools

In this example, we can use these tools.

In [4]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    return f"Starting music! {energetic=} {loud=}, {bpm=}"


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    return f"Lights are now set to {brightness}"

# Tool use

In langrila, you can define tool configs via ToolProperty, ToolParameter and ToolConfig object.

### ChatGPT

In [5]:
from langrila.openai import ToolConfig, ToolParameter, ToolProperty

chatgpt_tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="number",
                    description="The beats per minute of the music.",
                    enum=[60, 180],
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

ToolConfig object is internally converted to client tool configs

In [6]:
# You can see the formatted tool configs by running the following
chatgpt_tool_configs[1].format()

{'type': 'function',
 'function': {'name': 'start_music',
  'description': 'Play some music matching the specified parameters.',
  'parameters': {'type': 'object',
   'properties': {'energetic': {'type': 'boolean',
     'description': 'Whether the music is energetic or not.'},
    'loud': {'type': 'boolean',
     'description': 'Whether the music is loud or not.'},
    'bpm': {'type': 'number',
     'description': 'The beats per minute of the music.',
     'enum': [60, 180]}},
   'required': ['energetic', 'loud', 'bpm']}}}

In [7]:
chatgpt = ChatGPT(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=chatgpt_tool_configs,
)

In [8]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt)

# Response is CompletionResults object
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party mood is on! 🎉 The disco ball is spinning, energetic music is blasting at 180 BPM, and the lights are dimmed to create the perfect atmosphere. Let’s get this celebration started! 🕺💃')], name=None), usage=Usage(prompt_tokens=131, completion_tokens=47, total_tokens=178), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}], 'name': None}, {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_fVQu0irYgmHPHbyv4bVUcZ70', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_hfWWr3Tir4pDnkGpDKHPRNPn', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_9zH9yAmjzpj0dKMZ9YMl7T4Z', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type': 'function'}]}, {'role': 'tool', '

In [9]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is on! 🎉 The disco ball is spinning, energetic music is blasting at 180 BPM, and the lights are dimmed to create the perfect atmosphere. Let’s get this celebration started! 🕺💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 131,
  'completion_tokens': 47},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_fVQu0irYgmHPHbyv4bVUcZ70',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_hfWWr3Tir4pDnkGpDKHPRNPn',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_9zH9yAmjzpj0dKMZ9YMl7T4Z',
     'function': {'arguments': '{"brightnes

You can get the results from specific tool.

In [10]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt, tool_choice="start_music")

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The music is pumping at 180 BPM! 🎶 Let's add some colorful lights to set the vibe. How about some fun party games and snacks to keep the energy high? 🎉 What else do you want to include to make this party unforgettable?"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 60,
  'completion_tokens': 50},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_MHs9ys6WtyWIcnkffYU0ZO2V',
     'function': {'arguments': '{"energetic":true,"loud":true,"bpm":180}',
      'name': 'start_music'},
     'type': 'function'}]},
  {'role': 'tool',
   'tool_call_id': 'call_MHs9ys6WtyWIcnkffYU0ZO2V',
   'name': 'start_music',
   'content': 'Starting music! energetic=True loud=True, bpm=180'}]}

If you specify tool_only option as True, you get the FunctionCallingResults object instead of CompletionResults object.

In [11]:
response = chatgpt.run(prompt, tool_only=True)
response

FunctionCallingResults(usage=Usage(prompt_tokens=163, completion_tokens=75, total_tokens=238), results=[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_HPBYMNw2I2eLxEx55KsFT4M5', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'), Message(role='function', content=[ToolContent(output='Starting music! energetic=True loud=True, bpm=180', call_id='call_Gytf61Ijvtk8R1BXZ1yJUqre', args='{"energetic": true, "loud": true, "bpm": 180}', funcname='start_music')], name='start_music'), Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_fY8hhAU0VO9LIk4Faa2YmyXE', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')], calls=Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_HPBYMNw2I2eLxEx55KsFT4M5'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": 180}', call_id='call_Gytf61I

In [12]:
response.model_dump()

{'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 163,
  'completion_tokens': 75},
 'results': [{'role': 'function',
   'content': [{'output': 'Disco ball is spinning!',
     'call_id': 'call_HPBYMNw2I2eLxEx55KsFT4M5',
     'args': '{"power": true}',
     'funcname': 'power_disco_ball'}],
   'name': 'power_disco_ball'},
  {'role': 'function',
   'content': [{'output': 'Starting music! energetic=True loud=True, bpm=180',
     'call_id': 'call_Gytf61Ijvtk8R1BXZ1yJUqre',
     'args': '{"energetic": true, "loud": true, "bpm": 180}',
     'funcname': 'start_music'}],
   'name': 'start_music'},
  {'role': 'function',
   'content': [{'output': 'Lights are now set to 0.5',
     'call_id': 'call_fY8hhAU0VO9LIk4Faa2YmyXE',
     'args': '{"brightness": 0.5}',
     'funcname': 'dim_lights'}],
   'name': 'dim_lights'}],
 'calls': {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_HPBYMNw2I2eLxEx55KsFT4M5'},
  

In [13]:
# Tool result messages

response.results

[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_HPBYMNw2I2eLxEx55KsFT4M5', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'),
 Message(role='function', content=[ToolContent(output='Starting music! energetic=True loud=True, bpm=180', call_id='call_Gytf61Ijvtk8R1BXZ1yJUqre', args='{"energetic": true, "loud": true, "bpm": 180}', funcname='start_music')], name='start_music'),
 Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_fY8hhAU0VO9LIk4Faa2YmyXE', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')]

In [14]:
# Tool call messages

response.calls

Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_HPBYMNw2I2eLxEx55KsFT4M5'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": 180}', call_id='call_Gytf61Ijvtk8R1BXZ1yJUqre'), ToolCall(name='dim_lights', args='{"brightness": 0.5}', call_id='call_fY8hhAU0VO9LIk4Faa2YmyXE')], name=None)

### Gemini on GenerativeAI

In [6]:
from langrila.gemini.genai import ToolConfig, ToolParameter, ToolProperty

# NOTE : following tool_configs is almost same as that of OpenAI
gemini_tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "180", "240"],  # gemini does not support numerical enum
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [8]:
# You can see the formatted tool config as follows
gemini_tool_configs[1].format()

name: "start_music"
description: "Play some music matching the specified parameters."
parameters {
  type_: OBJECT
  properties {
    key: "loud"
    value {
      type_: BOOLEAN
      description: "Whether the music is loud or not."
    }
  }
  properties {
    key: "energetic"
    value {
      type_: BOOLEAN
      description: "Whether the music is energetic or not."
    }
  }
  properties {
    key: "bpm"
    value {
      type_: STRING
      description: "The beats per minute of the music."
      enum: "60"
      enum: "180"
      enum: "240"
    }
  }
  required: "energetic"
  required: "loud"
  required: "bpm"
}

In [9]:
gemini = Gemini(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=gemini_tool_configs,
)

In [10]:
prompt = "Turn this place into a party!"
response = gemini.run(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, everyone! Let's get this party started! The disco ball is going, the music is pumping, and the lights are low. Time to hit the dance floor! 🎉🥳 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 143,
  'completion_tokens': 38},
 'prompt': [parts {
    text: "Turn this place into a party!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "180"
          

Gemini allows you to call specific tool as well as ChatGPT.

In [11]:
prompt = "Turn this place into a party!"
response = gemini.run(prompt, tool_choice="start_music")

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, everyone! Let's get this party started!  Crank up the music! 🎉🥳 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 66,
  'completion_tokens': 21},
 'prompt': [parts {
    text: "Turn this place into a party!"
  }
  role: "user",
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "240"
          }
        }
      }
    }
  }
  role: "model",
  parts {
    function_response {
      name: "start_music"
      response {
        fields {
          key: "content"
          value {
            string_value: "Starting music! energetic=True loud=True, bpm=\'240\'"
          }
   

### Anthropic Claude

In [7]:
from langrila.claude import ToolConfig, ToolParameter, ToolProperty

# NOTE : following tool_configs is the same as that of OpenAI
claude_tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="number",
                    description="The beats per minute of the music.",
                    enum=[60, 180],
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [21]:
# You can see the formatted tool config as follows
claude_tool_configs[1].format()

{'name': 'start_music',
 'description': 'Play some music matching the specified parameters.',
 'input_schema': {'type': 'object',
  'properties': {'energetic': {'type': 'boolean',
    'description': 'Whether the music is energetic or not.'},
   'loud': {'type': 'boolean',
    'description': 'Whether the music is loud or not.'},
   'bpm': {'type': 'number',
    'description': 'The beats per minute of the music.',
    'enum': [60, 180]}},
  'required': ['energetic', 'loud', 'bpm']}}

In [22]:
claude = Claude(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=claude_tool_configs,
)

In [23]:
prompt = "Turn this place into a party!"

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! The party atmosphere is now set up:\n\n1. The lights have been dimmed to 30% brightness, creating a perfect party ambiance.\n2. Energetic and loud music is playing at 180 beats per minute, which is great for dancing and keeping the energy high.\n3. The disco ball is now spinning, adding that classic party vibe with its sparkling lights.\n\nIs there anything else you'd like to adjust for your party? Perhaps you want to change the music tempo, adjust the lighting, or turn off the disco ball at any point? Feel free to ask, and I'll be happy to help you fine-tune the party atmosphere!"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 976,
  'completion_tokens': 145},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a party!', 'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'text': "Certainly! I'd be happy to help turn this place into a party usi

# Multi-turn conversation with tools

In [10]:
from langrila import InMemoryConversationMemory, JSONConversationMemory

chatgpt = ChatGPT(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=chatgpt_tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [11]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is on! 🎉 The disco ball is spinning, energetic music is blasting at 180 BPM, and the lights are dimmed to create the perfect atmosphere. Let’s get this celebration started! 🕺💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 131,
  'completion_tokens': 47},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_NzLfAfO4lc3ahZB0WpSbTDfn',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_WZU4bJNMrDLuipC0JHfc0kuc',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_wemWgQwhm8JwqQvS9xV3b5NT',
     'function': {'arguments': '{"brightnes

Next turn

In [12]:
prompt = "Can you make bpm up more?"

response = chatgpt.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The music BPM has been cranked up to 240! 🎶 Get ready to dance even harder! Let the party vibes flow! 🕺💥'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 238,
  'completion_tokens': 32},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_NzLfAfO4lc3ahZB0WpSbTDfn',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_WZU4bJNMrDLuipC0JHfc0kuc',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_wemWgQwhm8JwqQvS9xV3b5NT',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'}]},
  {'role': 'tool',
   't

## Multi-turn conversation using tools with multiple model

In [15]:
from langrila import InMemoryConversationMemory

shared_memory = InMemoryConversationMemory()

chatgpt = ChatGPT(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=chatgpt_tool_configs,
    conversation_memory=shared_memory,
)

gemini = Gemini(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=gemini_tool_configs,
    conversation_memory=shared_memory,
)

In [16]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt)

In [17]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is on! 🎉 The disco ball is spinning, energetic music is blasting at 180 BPM, and the lights are dimmed to create the perfect atmosphere. Let’s get this celebration started! 🕺💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 131,
  'completion_tokens': 47},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_hexZONJYJpbDkqPYKTDnoYwp',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_tFRoJUAmIWyVacSfWFSFEGpN',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_NqcdKre7cUhYOZtOnJ0Fa26J',
     'function': {'arguments': '{"brightnes

Gemini can continue conversation without any additional process.

In [18]:
prompt = "Can you make bpm up more?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The music is pumping even faster now!  We're at a solid 240 BPM, getting everyone energized. Time to really get moving! ⚡️ 🎶 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 257,
  'completion_tokens': 35},
 'prompt': [parts {
    text: "Turn this place into a party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            number_value: 180
          }
        }
      }
    

Gemini also works as well.

NOTE : Claude may not work in multi-turn conversation using tools. Bug fix is in progress.

# Multi-turn conversation with tools and context truncation

Although context truncation doesn't work accurately if we use tools, but truncation is useful for token management to some extent.

In [25]:
from langrila import InMemoryConversationMemory

chatgpt = ChatGPT(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=chatgpt_tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    context_length=50,
)

In [26]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt, tool_choice="auto")

In [27]:
prompt = "Can you make bpm up more?"

response = chatgpt.run(prompt=prompt, tool_choice="auto")

Input message is truncated because total length of messages exceeds context length.
Input message is truncated because total length of messages exceeds context length.


User message and assistant message are target of truncation while tool call message and tool result message is not.

In [30]:
print(response.message.content[0].text)

The music BPM has been increased to 240! Get ready to dance even harder! 🎶💥


In [31]:
response.usage

Usage(prompt_tokens=208, completion_tokens=21, total_tokens=229)

In [28]:
response.prompt

[{'content': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': [{'id': 'call_4YGGp1QGzQeTV9uIyWukUlvB',
    'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
    'type': 'function'},
   {'id': 'call_oAbGUbgCnNOZYFW5dxwLYfqR',
    'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
     'name': 'start_music'},
    'type': 'function'},
   {'id': 'call_2eLJhoAbRR0o7lq8T9QVUBdS',
    'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_4YGGp1QGzQeTV9uIyWukUlvB',
  'name': 'power_disco_ball',
  'content': 'Disco ball is spinning!'},
 {'role': 'tool',
  'tool_call_id': 'call_oAbGUbgCnNOZYFW5dxwLYfqR',
  'name': 'start_music',
  'content': 'Starting music! energetic=True loud=True, bpm=180'},
 {'role': 'tool',
  'tool_call_id': 'call_2eLJhoAbRR0o7lq8T9QVUBdS',
  'name': 'dim_lights',
  'content': 'Lights are now set to 0.5'},
 {'role': 'ass

In [29]:
# whole history
chatgpt.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_4YGGp1QGzQeTV9uIyWukUlvB'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": 180}',
    'call_id': 'call_oAbGUbgCnNOZYFW5dxwLYfqR'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_2eLJhoAbRR0o7lq8T9QVUBdS'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_4YGGp1QGzQeTV9uIyWukUlvB',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Starting music! energetic=True loud=True, bpm=180',
    'call_id': 'call_oAbGUbgCnNOZYFW5dxwLYfqR',
    'args': '{"energetic": true, "loud": true, "bpm": 180}',
    'funcname': 'start_music'}],
  'name': None},
 {'role': 'function'